In [7]:
import sys
sys.modules['__main__'].__file__ = None  # add-hoc for disable Pony interactive mode.

#sys.path.append('E:\\learn\\CGRdb')

import pony.orm
from pony.orm import db_session, select, Database

import CGRtools
from CGRtools import RDFRead, ReactionContainer, SDFRead, SMILESRead, SDFWrite

import CGRdb
from CGRdb import load_schema, Reaction, Molecule
from CGRdbData import MoleculeClass, MoleculeProperties


In [8]:
CGRtools.containers.molecule.MoleculeContainer.

Help on class MoleculeContainer in module CGRtools.containers.molecule:

class MoleculeContainer(CGRtools.algorithms.stereo.MoleculeStereo, CGRtools.containers.common.Graph, CGRtools.algorithms.aromatics.Aromatize, CGRtools.algorithms.standardize.molecule.Standardize, CGRtools.algorithms.smiles.MoleculeSmiles, CGRtools.algorithms.components.structure.StructureComponents, CGRtools.algorithms.depict.DepictMolecule, CGRtools.algorithms.calculate2d.Calculate2DMolecule, CGRtools.algorithms.tautomers.Tautomers, CGRtools.algorithms.huckel.Huckel, CGRtools.algorithms.x3dom.X3domMolecule)
 |  SSSR calculation. based on idea of PID matrices from:
 |  Lee, C. J., Kang, Y.-M., Cho, K.-H., & No, K. T. (2009).
 |  A robust method for searching the smallest set of smallest rings with a path-included distance matrix.
 |  Proceedings of the National Academy of Sciences of the United States of America, 106(41), 17355–17358.
 |  http://doi.org/10.1073/pnas.0813040106
 |  
 |  Method resolution order:
 | 

Прочитаем молекулы из SDF файла, затем получим их метаданные

# Чтение файлов

Прочитаем SDF файл

In [2]:
aa = SDFRead('output0.sdf').read()

In [5]:
type(aa[1])

CGRtools.containers.molecule.MoleculeContainer

In [4]:
aa[3465].meta

{'SMILES': 'Cl[Au](Cl)Cl', 'catNo': '12-1430'}

Прочитаем из файла смайлсы, обрежем номера и запишем чистые смайлсы в другой файл (можно не делать, так как файл уже готов)

In [90]:
smls = []
# прочитаем из файла
with open('sial.src.txt', 'r') as f:
    for line in f:
        line = line.split()[0] # оставим только смайлс
        smls.append(line)
        
# запишем в файл 
with open('my_smiles', 'w') as f1:
    for i in smls[:100]: # запишем только сотню смайлсов для демонстрации
        f1.write(i + '\n')

Прочитаем файл чистых смайлс и сделаем из него набор CGR-контейнеров

In [91]:
ss = SMILESRead('my_smiles').read()

# Соединение с базой данных

Будет печатать pony запросы в формате SQL, если включить дебаг

In [92]:
pony.orm.set_sql_debug(True)

Присоединимся к базе данных:

In [3]:
db = load_schema('schema_name', password='secret', port=5432, host='postgredb_service', user='postgres')

/usr/local/lib/python3.8/site-packages/LazyPony/__init__.py:88: UserWarning: for many-to-many relationship if schema used NEED to define m2m table name
  warn('for many-to-many relationship if schema used NEED to define m2m table name')


# Добавление молекул в базу данных

Выберем случайную молекулу из сета

In [9]:
i = 6
ss1 = ss[i]
ss1.clean2d() # для отрисовки молекулы
ss1

Добавим одну молекулу(команда insert), если такая молекула уже есть, то выдаст ошибку(TransactionIntegrityError)

In [10]:
with db_session:
    m = Molecule(ss1)

GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':1, 'p2':b'\x80\x04\x95I\x05\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x00K\x02K\x00K\x03K\x01K\x04K\x01K\x05K\x00K\x06K\x00K\x07K\x02K\x08K\x02K\tK\x02K\nK\x00K\x0bK\x00u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c CGRtools.periodictable.groupXVII\x94\x8c\x01I\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x03h\x19)\x81\x94}\x94h\x16NsbK\x04h\x19)\x81\x94}\x94h\x16NsbK\x05h\x19)\x81\x94}\x94h\x16NsbK\x06\x8c\x1fCGRtools.periodictable.groupXVI\x94\x8c\x01O\x94\x93\x94)\x81

Посмотрим, какие есть атрибуты у нашей молекулы(у таблицы Молекула)

In [11]:
pony.orm.show(Molecule)

class Molecule(Entity):
    id = PrimaryKey(int, auto=True)
    _structures = Set(MoleculeStructure)
    _reactions = Set(MoleculeReaction)
    metadata = Set(MoleculeProperties)
    classes = Set(MoleculeClass)


Наполним БД молекулами из нашего множества

In [12]:
for molecule in ss:   # example of multiple molecules storing. for reactions same approach.
    try:    
        with db_session:
            m = Molecule(molecule)
    except:
        pass

GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':2, 'p2':b'\x80\x04\x95\xd4\x04\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x01K\x02K\x00K\x03K\x01K\x04K\x00K\x05K\x02K\x06K\x02K\x07K\x02K\x08K\x00K\tK\x00K\nK\x01u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02\x8c\x1eCGRtools.periodictable.groupXV\x94\x8c\x01N\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x03h\x13)\x81\x94}\x94h\x16NsbK\x04h\x13)\x81\x94}\x94h\x16NsbK\x05h\x13)\x81\x94}\x94h\x16NsbK\x06h\x13)\x81\x94}\x94h\x16NsbK\x07h\x13)\x81\x94}\x94h\x16NsbK\x08\x8c\x1fCGR

COMMIT
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':23, 'p2':b'\x80\x04\x95\xd4\x07\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x03K\x02K\x00K\x03K\x03K\x04K\x00K\x05K\x00K\x06K\x00K\x07K\x00K\x08K\x00K\tK\x02K\nK\x02K\x0bK\x00K\x0cK\x00K\rK\x00K\x0eK\x01K\x0fK\x01K\x10K\x00K\x11K\x00K\x12K\x03K\x13K\x03u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02h\x13)\x81\x94}\x94h\x16NsbK\x03h\x13)\x81\x94}\x94h\x16NsbK\x04\x8c\x1fCGRtools.periodictable.groupXVI\x94\x8c\x01O\x94\x93\x94)\x81\x94}\x94h\x1

COMMIT
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':42, 'p2':b"\x80\x04\x95J\x05\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x00K\x02K\x00K\x03K\x01K\x04K\x00K\x05K\x00K\x06K\x00K\x07K\x02K\x08K\x02K\tK\x00K\nK\x00K\x0bK\x01u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c CGRtools.periodictable.groupXVII\x94\x8c\x02Cl\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x03h\x19)\x81\x94}\x94h\x16NsbK\x04\x8c\x1eCGRtools.periodictable.groupXV\x94\x8c\x01N\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x05h\x19)\x81\x94}

COMMIT
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':62, 'p2':b'\x80\x04\x95r\x05\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x02K\x02K\x00K\x03K\x00K\x04K\x01K\x05K\x00K\x06K\x00K\x07K\x01K\x08K\x00K\tK\x00K\nK\x01K\x0bK\x00K\x0cK\x00u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c\x1eCGRtools.periodictable.groupXV\x94\x8c\x01N\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x03h\x13)\x81\x94}\x94h\x16NsbK\x04h\x19)\x81\x94}\x94h\x16NsbK\x05h\x13)\x81\x94}\x94h\x16NsbK\x06h\x19)\x81\x94}\x94h\x16NsbK\x0

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':80, 'p2':b'\x80\x04\x95\x88\x06\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x03K\x02K\x00K\x03K\x03K\x04K\x03K\x05K\x00K\x06K\x00K\x07K\x00K\x08K\x00K\tK\x01K\nK\x00K\x0bK\x00K\x0cK\x00K\rK\x00K\x0eK\x00K\x0fK\x00u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02h\x13)\x81\x94}\x94h\x16NsbK\x03h\x13)\x81\x94}\x94h\x16NsbK\x04h\x13)\x81\x94}\x94h\x16NsbK\x05h\x13)\x81\x94}\x94h\x16NsbK\x06\x8c\x1eCGRtools.periodictable.groupXV\x94\x8c\x01N\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x07h\x13)\x81\x94}\x94h\x16NsbK\x08h\x13)\x81\x94}\x94h\x16NsbK\th\x13)\x81\x94}\x94h\x16NsbK\nh\x13

COMMIT
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
INSERT INTO "schema_name"."Molecule" DEFAULT VALUES RETURNING "id"

INSERT INTO "schema_name"."MoleculeStructure" ("molecule", "structure") VALUES (%(p1)s, %(p2)s) RETURNING "id"
{'p1':96, 'p2':b'\x80\x04\x95m\x04\x00\x00\x00\x00\x00\x00\x8c\x1cCGRtools.containers.molecule\x94\x8c\x11MoleculeContainer\x94\x93\x94)\x81\x94}\x94(\x8c\nconformers\x94]\x94\x8c\thydrogens\x94}\x94(K\x01K\x02K\x02K\x00K\x03K\x00K\x04K\x01K\x05K\x00K\x06K\x01K\x07K\x00K\x08K\x01K\tK\x01u\x8c\x0catoms_stereo\x94}\x94\x8c\x0eallenes_stereo\x94}\x94\x8c\x10cis_trans_stereo\x94}\x94\x8c\x05atoms\x94}\x94(K\x01\x8c\x1eCGRtools.periodictable.groupXV\x94\x8c\x01N\x94\x93\x94)\x81\x94}\x94\x8c\x07isotope\x94NsbK\x02\x8c\x1fCGRtools.periodictable.groupXIV\x94\x8c\x01C\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x03h\x19)\x81\x94}\x94h\x16NsbK\x04h\x19)\x81\x94}\x94h\x16NsbK\x05\x8c\x1fCGRtools.periodictable.groupXVI\x94\x8c\x01O\x94\x93\x94)\x81\x94}\x94h\x16NsbK\x06

# Извлечение молекул из базы данных

Поиграемся с pony запросами, вот так делается селект.

In [93]:
with db_session:
    a = select(m for m in Molecule if m.id > 10 and m.id < 50)[:] # Скобки в конце нужны, чтобы превратить запрос в список

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "m"."id"
FROM "schema_name"."Molecule" "m"
WHERE "m"."id" > 10
  AND "m"."id" < 50

RELEASE CONNECTION


Который вернет нам список из молекул

In [94]:
a

[Molecule[11], Molecule[12], Molecule[13], Molecule[14], Molecule[15], Molecule[16], Molecule[17], Molecule[18], Molecule[19], Molecule[20], Molecule[21], Molecule[22], Molecule[23], Molecule[24], Molecule[25], Molecule[26], Molecule[27], Molecule[28], Molecule[29], Molecule[30], Molecule[31], Molecule[32], Molecule[33], Molecule[34], Molecule[35], Molecule[36], Molecule[37], Molecule[38], Molecule[39], Molecule[40], Molecule[41], Molecule[42], Molecule[43], Molecule[44], Molecule[45], Molecule[46], Molecule[47], Molecule[48], Molecule[49]]

Если мы хотим посмотреть структуру молекулы, например третьей из нашего списка, то есть молекулу с id=13, то надо снова подключаться к базе данных и вытаскивать информацию оттуда 

In [95]:
with db_session:
    struc = a[3].structure

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "ms"."id", "ms"."molecule", "ms"."is_canonic", "ms"."structure"
FROM "schema_name"."MoleculeStructure" "ms"
WHERE "ms"."molecule" = %(p1)s
  AND "ms"."is_canonic"
ORDER BY 1
LIMIT 1
{'p1':14}

RELEASE CONNECTION


In [96]:
struc.clean2d() # для отрисовки
struc

## Поиск по структуре

In [97]:
with db_session:
    found = Molecule.find_structure(struc)
    fs = found.structure 

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495b2060000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b004b044b004b054b004b064b034b074b03758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014f9493942981947d948c0769736f746f7065944e73624b028c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d9468164e73624b0368192981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b0668192981947d9468164e73624b0768192981947d9468164e7362758c05626f6e6473947d94284b017d94284b028c19434752746f6f6c732e636f6e7461696e6572732e626f6e6473948c04426f6e64949394298194

In [98]:
fs.clean2d()
fs

## Поиск подструктур

In [99]:
with db_session:
    found = Molecule.find_substructures(struc)  # MoleculeSearchCache object or None returned

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_substructure_molecules('\x800495b2060000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b004b044b004b054b004b064b034b074b03758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014f9493942981947d948c0769736f746f7065944e73624b028c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d9468164e73624b0368192981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b0668192981947d9468164e73624b0768192981947d9468164e7362758c05626f6e6473947d94284b017d94284b028c19434752746f6f6c732e636f6e7461696e6572732e626f6e6473948c04426f6e6494939429

В результате получим объект, который содержит все подструктуры

In [100]:
found, len(found), found.operator

(MoleculeSearchCache[1], 1, 'substructure')

Превратим его в список

In [101]:
with db_session:
    molecules = found.molecules(1, 10)  # возьмем первые 10

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "x"."molecules"[case when %(p1)s >= 0 then (%(p1)s + 1) else (COALESCE(ARRAY_LENGTH("x"."molecules", 1), 0) + (%(p1)s + 1)) end:case when %(p4)s >= 0 then %(p4)s else (COALESCE(ARRAY_LENGTH("x"."molecules", 1), 0) + %(p4)s) end]
FROM "schema_name"."MoleculeSearchCache" "x"
WHERE "x"."id" = %(p7)s
ORDER BY 1
LIMIT 1
{'p1':0, 'p4':10, 'p7':1}

SELECT "x"."id"
FROM "schema_name"."Molecule" "x"
WHERE "x"."id" IN (%(p1)s)
{'p1':14}

SELECT "x"."id", "x"."molecule", "x"."is_canonic", "x"."structure"
FROM "schema_name"."MoleculeStructure" "x"
WHERE "x"."molecule" IN (%(p1)s)
  AND "x"."is_canonic"
{'p1':14}

RELEASE CONNECTION


In [102]:
molecules

[Molecule[14]]

Посмотрим на две получившиеся структуры

In [104]:
with db_session:
    mol1 = molecules[0].structure
    #mol2 = molecules[1].structure

In [105]:
mol1.clean2d()
mol1

In [25]:
mol2.clean2d()
mol2

NameError: name 'mol2' is not defined

In [107]:
struc

## Поиск по подобию

In [106]:
with db_session:
    found = Molecule.find_similar(struc)  # MoleculeSearchCache object or None returned
found

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_similar_molecules('\x800495b2060000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b004b044b004b054b004b064b034b074b03758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014f9493942981947d948c0769736f746f7065944e73624b028c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d9468164e73624b0368192981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b0668192981947d9468164e73624b0768192981947d9468164e7362758c05626f6e6473947d94284b017d94284b028c19434752746f6f6c732e636f6e7461696e6572732e626f6e6473948c04426f6e649493942981944

MoleculeSearchCache[2]

In [108]:
found, len(found), found.operator

(MoleculeSearchCache[2], 3, 'similar')

In [109]:
with db_session:
    molecules = found.molecules(1, 10)  # возьмем первые 10

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "x"."molecules"[case when %(p1)s >= 0 then (%(p1)s + 1) else (COALESCE(ARRAY_LENGTH("x"."molecules", 1), 0) + (%(p1)s + 1)) end:case when %(p4)s >= 0 then %(p4)s else (COALESCE(ARRAY_LENGTH("x"."molecules", 1), 0) + %(p4)s) end]
FROM "schema_name"."MoleculeSearchCache" "x"
WHERE "x"."id" = %(p7)s
ORDER BY 1
LIMIT 1
{'p1':0, 'p4':10, 'p7':2}

SELECT "x"."id"
FROM "schema_name"."Molecule" "x"
WHERE "x"."id" IN (%(p1)s, %(p2)s, %(p3)s)
{'p1':14, 'p2':188, 'p3':367}

SELECT "x"."id", "x"."molecule", "x"."is_canonic", "x"."structure"
FROM "schema_name"."MoleculeStructure" "x"
WHERE "x"."molecule" IN (%(p1)s, %(p2)s, %(p3)s)
  AND "x"."is_canonic"
{'p1':14, 'p2':188, 'p3':367}

RELEASE CONNECTION


In [110]:
molecules

[Molecule[14], Molecule[188], Molecule[367]]

In [119]:
with db_session:
    structures = [mol.structure for mol in molecules]
structures

In [120]:
for m in structures:
    m.clean2d()    

Видно, что молекулы расположены в порядке убывания коэффициента Танимто

In [113]:
with db_session:
    tnmt = found.tanimotos()

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "x"."tanimotos"[case when %(p1)s >= 0 then (%(p1)s + 1) else (COALESCE(ARRAY_LENGTH("x"."tanimotos", 1), 0) + (%(p1)s + 1)) end:case when %(p4)s >= 0 then %(p4)s else (COALESCE(ARRAY_LENGTH("x"."tanimotos", 1), 0) + %(p4)s) end]
FROM "schema_name"."MoleculeSearchCache" "x"
WHERE "x"."id" = %(p7)s
ORDER BY 1
LIMIT 1
{'p1':0, 'p4':100, 'p7':2}

RELEASE CONNECTION


In [114]:
tnmt, len(tnmt)

([1.0, 0.53571427, 0.5060241], 3)

In [121]:
struc

In [122]:
structures[0] # та же самая молекула

In [123]:
structures[1]

In [124]:
structures[2]

In [39]:
structures[9]

IndexError: list index out of range

In [40]:
aa[0].meta

{'SMILES': 'COC1=CC=C(C2=CC(=O)C3=C(C=C(O)C=C3O)O2)C=C1', 'catNo': '00017'}

# Добавление наркотиков в БД

здесь используем cgrdbDATA для того, чтобы извлекать метаданные и добавлять в базу 


Прочитаем наркотики из файла с наркотиками

In [125]:
nrktk = SDFRead('Narkotiki_test.sdf').read()
len(nrktk)

578

Посмотрим метаданные для случайной молекулы и ее стукртуру, чтобы убедиться, что все работает правильно

In [126]:
nrktk[16].meta

{'ID': '17',
 'references': '218. Е. А. Симонов, Л. Ф. Найденова, С. А. Ворнаков. Наркотические средства и психотропные вещества, контролируемые на территории РФ. М., 2003, стр. 40, 41',
 'UPAC_name': '2-(diethylamino)ethyl 2,2-diphenylpropanoate',
 'CAS': '3563-01-7',
 'List': '3',
 'category': 'психотропные вещества',
 'DateADD': '30.06.1998',
 'NumberPOST': '681',
 'NameRUS': 'Апрофен',
 'NameTRIVIAL': '[2-(диэтиламино)этил]-2,2-дифенилпропаноат',
 'Synonym1': 'Aprophen, Aprofen'}

In [7]:
nrktk[16]

Создадим класс молекул, который, назовем 'nrktk', и добавим ключ(_type), по которому можно удобно обращаться к этому классу

In [8]:
with db_session:
    db.MoleculeClass(name='nrktk', _type=1)

Проверим, создался ли класс

In [9]:
with db_session:
    c = MoleculeClass[1]
c.name

'nrktk'

Добавим молекулы из списка наркотиков вместе с метаданными. Минус кода внизу в том, что он не будет дописывать метаданные к существующим, а перетрёт их теми, которые мы дадим

In [10]:
for molecule in nrktk:
    with db_session:
        
        if not Molecule.structure_exists(molecule): # проверим, существует ли данная молекула в базе данных
            m = Molecule(molecule)                  # добавим молекулу в базу
        else:                                       # если такая молекула уже есть
            m = Molecule.find_structure(molecule)   # то найдем эту молекулу
       
        meta = molecule.meta                        # возьмем метаданные
        MoleculeProperties(structure=m, data=meta)  # сохраним их в молекулу
        m.classes = [MoleculeClass[1]]              # добавим класс в молекулу, в нашем случае, это созданный 'nrktk'


Посмотрим, что у нас есть в базе меткой наркотиков

In [11]:
with db_session:
    # загрузим класс 'nrktk'
    nrktk_class = MoleculeClass.select(lambda c: c.name=='nrktk')[:][0] 
    # Превратим этот запрос в список
    db_nrktk = list(nrktk_class.structures)
    # Схраним все метаданные в список
    metadata = [mol.metadata.data for mol in db_nrktk]
db_nrktk[:5]

[Molecule[4], Molecule[102], Molecule[238], Molecule[5], Molecule[239]]

In [4]:
metadata

NameError: name 'metadata' is not defined

Проверим, все ли правильно работает

In [49]:
len(db_nrktk), len(nrktk) # видим, что число наркотиков в базе данных и в файле не совпадает

(572, 578)

In [50]:
i = 0 # заведем счетчики всех молекул и несуществующих
j = 0
for molecule in nrktk:
    with db_session:
        if not Molecule.structure_exists(molecule):
            j+=1
    i+=1

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x8004957b0a0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b014b034b024b044b004b054b004b064b024b074b004b084b004b094b024b0a4b004b0b4b014b0c4b024b0d4b024b0e4b034b0f4b024b104b014b114b014b124b034b134b014b144b014b154b01758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b048c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014f9493942981947d9468164e73624b058c1e434752746f6f6c732e706572696f6469637461626c652e67726f75705856948c014e9493942981947d

GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x80049560070000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b004b044b004b054b014b064b024b074b014b084b014b094b014b0a4b01758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b048c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014f9493942981947d9468164e73624b05681d2981947d9468164e73624b068c1e434752746f6f6c732e706572696f6469637461626c652e67726f75705856948c014e9493942981947d9468164e73624b0768132981947d9468164e73624b0868132981947d9468

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495210d0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b014b044b004b054b004b064b004b074b014b084b004b094b014b0a4b024b0b4b004b0c4b014b0d4b014b0e4b024b0f4b004b104b004b114b004b124b024b134b014b144b004b154b014b164b004b174b034b184b034b194b034b1a4b00758c0c61746f6d735f73746572656f947d94284b03894b01894b0c894b04884b0789758c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b068c1f434752746f6f6c732e706572696f646963746

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495bd0b0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b014b044b004b054b004b064b004b074b014b084b004b094b024b0a4b004b0b4b024b0c4b004b0d4b024b0e4b004b0f4b024b104b014b114b024b124b004b134b014b144b004b154b034b164b034b174b00758c0c61746f6d735f73746572656f947d94284b03894b01894b05884b0789758c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b068c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c0

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495690b0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b004b024b004b034b004b044b014b054b004b064b004b074b014b084b004b094b024b0a4b004b0b4b024b0c4b024b0d4b004b0e4b014b0f4b024b104b014b114b024b124b014b134b014b144b014b154b034b164b00758c0c61746f6d735f73746572656f947d94284b04894b01894b0e894b05884b0789758c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b068c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570585649948c014

...

SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x80049525060000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b014b024b014b034b014b044b014b054b004b064b014b074b024b084b024b094b024b0a4b014b0b4b03758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b0668132981947d9468164e73624b0768132981947d9468164e73624b0868132981947d9468164e73624b0968132981947d9468164e73624b0a8c1e434752746f6f6c732e706572696f6469637461626c652e67726f75705856948c014e9493942981947d9468164e73624b0b68132981947d9468164e7362758c05626f6

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x8004956e0c0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b024b024b004b034b024b044b024b054b014b064b024b074b024b084b024b094b004b0a4b004b0b4b014b0c4b014b0d4b014b0e4b004b0f4b004b104b004b114b034b124b034b134b004b144b004b154b004b164b004b174b024b184b034b194b014b1a4b014b1b4b014b1c4b014b1d4b01758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b028c1e434752746f6f6c732e706572696f6469637461626c652e67726f75705856948c014e9493942981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495420c0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b024b024b004b034b024b044b024b054b014b064b024b074b004b084b004b094b004b0a4b004b0b4b004b0c4b014b0d4b014b0e4b014b0f4b014b104b014b114b014b124b014b134b014b144b014b154b014b164b024b174b024b184b004b194b014b1a4b014b1b4b014b1c4b014b1d4b01758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b028c1e434752746f6f6c732e706572696f6469637461626c652e67726f75705856948c014e9493942981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132

RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT * FROM "schema_name".cgrdb_search_structure_molecule('\x800495b70a0000000000008c1c434752746f6f6c732e636f6e7461696e6572732e6d6f6c6563756c65948c114d6f6c6563756c65436f6e7461696e65729493942981947d94288c0a636f6e666f726d657273945d948c09687964726f67656e73947d94284b014b024b024b004b034b014b044b014b054b014b064b024b074b004b084b034b094b024b0a4b034b0b4b004b0c4b004b0d4b014b0e4b004b0f4b004b104b004b114b014b124b004b134b004b144b024b154b024b164b024b174b024b184b03758c0c61746f6d735f73746572656f947d948c0e616c6c656e65735f73746572656f947d948c106369735f7472616e735f73746572656f947d948c0561746f6d73947d94284b018c1f434752746f6f6c732e706572696f6469637461626c652e67726f7570584956948c01439493942981947d948c0769736f746f7065944e73624b0268132981947d9468164e73624b0368132981947d9468164e73624b0468132981947d9468164e73624b0568132981947d9468164e73624b0668132981947d9468164e73624b0768132981947d9468164e73624b0868132981947d9468164e73624b096813298

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
i, j # нет несуществующих молекул, значит какие-то молекулы в файле повторяются

(578, 0)

Обеспечим правильный ввод смайлз для поиска по подобию.

In [85]:
a = CGRtools.smiles('c4cccc3c4ccc2c3ccc1c2cccc1')
if not isinstance(a, CGRtools.containers.molecule.MoleculeContainer):
    raise ValueError('Incorrect SMILES')
a.clean2d()
a

In [ ]:
CGRtools.

In [52]:
type(a)

CGRtools.containers.molecule.MoleculeContainer